In [21]:
import warnings

# Ignore warning messages during code execution
# The warnings module is imported
# The .filterwarnings() function is called on the warnings module
# The 'ignore' argument is passed to the .filterwarnings() function
# It configures the warnings module to ignore warning messages

warnings.filterwarnings('ignore')


In [22]:
import geopandas as gpd
#from folium import Markup


In [23]:
import geopandas as gpd

# Read the GeoJSON file and load its contents into a GeoDataFrame
gdf = gpd.read_file("dummy_data.geojson")

# Display the first few rows of the GeoDataFrame
gdf.tail()


,_id,sno,unique_id,coordinates,gis,plus_code,nearby_location,photos,videos,type_of_item,status,latitude,longitude,geometry
0,64bc1f3c1051ae135cc76ab7,1,XY124,"-1.777752,36.924870","1°46'40.5""""S 36°55'29.5""""E",5Q5V+22 Kajiado,KAJIADO RAILWAY STATION,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFh...,AAAAGGZ0eXBtcDQyAAAAAGlzb21tcDQyAAAQe21vb3YAAA...,B,5,-1.777752,36.924870,POINT (36.92487 -1.77775)
1,64bcb4a50949ddf4f3eed98f,2,XX127,"-4.04348,39.668206","4°02'36.5""""S 39°40'05.5""""E",XHHH+6M Mombasa,MOMBASA TERMINUS,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBUVFRgWFR...,AAAAGGZ0eXBtcDQyAAAAAGlzb21tcDQyAAAQw21vb3YAAA...,B,3,-4.043480,39.668206,POINT (39.66821 -4.04348)


In [24]:
import folium
import pandas as pd
import base64

# Sample GeoDataFrame with data
# Assuming you already have 'gdf' DataFrame as shown in the dataset

# Define a dictionary to map each type to a color
type_colors = {
    'A': 'blue',
    'B': 'green',
    'C': 'red'
}

# Create a map object using the GeoDataFrame
m = folium.Map()

# Define a function to create the HTML content for the pop-up pane
def create_popup_html(row):
    unique_id = row['unique_id']
    coordinates = row['coordinates']
    gis = row['gis']
    plus_code = row['plus_code']
    image_base64 = row['photos']
    
    # Specify the base64 encoded image source
    image_src = f'data:image/jpeg;base64,{image_base64}'
    
    popup_html = f'''
        <img src="{image_src}" alt="Image" width="200"><br>
        <b>Unique ID:</b> {unique_id}<br>
        <b>Coordinates:</b> {coordinates}<br>
        <b>GIS:</b> {gis}<br>
        <b>PLUS CODE:</b> {plus_code}<br>
        <a href="#" onclick="toggleNavigationPane('{unique_id}', event)">View More Details</a>
    '''

    return popup_html

# Define the CSS and JavaScript code for the navigation pane
css_navigation_pane = '''
    <style>
    /* Custom CSS for the navigation pane */
    .navigation-ui {
        display: none;
        position: absolute;
        top: 80px; /* Adjust the top position as needed */
        left: 10px;
        padding: 10px;
        background-color: white;
        z-index: 1000;
    }
    .video-container {
        margin-bottom: 10px;
    }
    </style>
'''

js_navigation_pane = '''
    <script>
    // JavaScript code for the navigation pane
    function toggleNavigationPane(uniqueId, event) {
        event.preventDefault();
        var navigationPane = document.getElementById('navigation-ui-' + uniqueId);
        if (navigationPane.style.display === 'none') {
            navigationPane.style.display = 'block';
        } else {
            navigationPane.style.display = 'none';
        }
    }
    </script>
'''

# Function to generate custom HTML for the navigation pane
def generate_navigation_ui(row):
    unique_id = row['unique_id']
    type_of_item = row['type_of_item']
    status = row['status']
    video_base64 = row['videos']
    
    # Specify the base64 encoded video source
    video_src = f'data:video/mp4;base64,{video_base64}'
    
    ui_html = f'''
        <div id="navigation-ui-{unique_id}" class="navigation-ui">
            <h2>Navigation UI</h2>
            <p><b>Unique ID:</b> {unique_id}</p>
            <p><b>Type of item:</b> {type_of_item}</p>
            <p><b>Status:</b> {status}</p>
            <div class="video-container">
                <video width="320" height="240" controls>
                    <source src="{video_src}" type="video/mp4">
                    Your browser does not support the video tag.
                </video>
            </div>
        </div>
    '''
    
    return ui_html

# Create the navigation pane HTML content
selected_columns = ['unique_id', 'type_of_item', 'status', 'videos']
selected_gdf = gdf[selected_columns]

# Iterate over the rows of the GeoDataFrame
for index, row in gdf.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    type_of_item = row['type_of_item']
    
    # Get the color for the marker based on the type
    marker_color = type_colors.get(type_of_item, 'gray')

    popup_html = create_popup_html(row)

    marker = folium.Marker(location=[latitude, longitude], popup=folium.Popup(popup_html),
                           icon=folium.Icon(color=marker_color), sticky=True)
    marker.add_to(m)

    navigation_ui_html = generate_navigation_ui(row)
    navigation_ui_element = folium.Element(navigation_ui_html)
    m.get_root().html.add_child(navigation_ui_element)

    # Add a script tag to hide the navigation pane initially
    m.get_root().html.add_child(folium.Element(f"<script>document.getElementById('navigation-ui-{row['unique_id']}').style.display = 'none';</script>"))

# Add the CSS and JavaScript code to the map
m.get_root().html.add_child(folium.Element(css_navigation_pane))
m.get_root().html.add_child(folium.Element(js_navigation_pane))

# Save the map to an HTML file
m.save('gis/templates/map.html')


# Handle image data
#read images and convert them base64
import pandas as pd
import base64

# Function to read and convert an image to base64
def image_to_base64(file_path):
    with open(file_path, 'rb') as f:
        image_data = f.read()
    base64_image = base64.b64encode(image_data).decode('utf-8')
    return base64_image

# List of image file paths
image_files = ['C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\junagarh_junction.jpeg',
               'C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\nairobi.jpeg',
               'C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\imara_daima.jpeg',
               'C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\syokimau.jpeg',
               'C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\kisumu.jpeg'
               ]

# Convert each image to base64 and store in a DataFrame
image_data = []
for file_path in image_files:
    base64_image = image_to_base64(file_path)
    image_data.append(base64_image)

df = pd.DataFrame({'Image': image_data})

# Print the DataFrame
print(df)

#Add the df records to gdf

# Assuming you have already created the 'gdf' DataFrame with a 'Location_ID' column
# and the 'pd' DataFrame with an 'Image' column containing base64 encoded images

# Loop through 'df' DataFrame and populate 'photos' column in 'gdf'
for index, row in df.iterrows():
    image_base64 = row['Image']
    gdf.at[index, 'photos'] = image_base64

# Now the 'photos' column in the 'gdf' DataFrame will be filled with base64 encoded images from the 'df' DataFrame

gdf.head()

#Handle Video data
import pandas as pd
import base64

# Function to read and convert a video to base64
def video_to_base64(file_path):
    with open(file_path, 'rb') as f:
        video_data = f.read()
    base64_video = base64.b64encode(video_data).decode('utf-8')
    return base64_video

# List of video file paths
video_files = ['C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\junagarh_junction.mp4',
               'C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\nairobi.mp4',
               'C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\imara_daima.mp4',
               'C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\syokimau.mp4',
               'C:\\Users\\emm14\\OneDrive\\Desktop\\Controlytics project\\navigation pane\\kisumu.mp4'
               ]

# Convert each video to base64 and store in a DataFrame
video_data = []
for file_path in video_files:
    base64_video = video_to_base64(file_path)
    video_data.append(base64_video)

# Assuming you have already created the 'pd' DataFrame with 'Video' column
# If not, you can create it like this:
pd_data = {'Video': video_data}
pd_df = pd.DataFrame(pd_data)

# Print the DataFrame
print(pd_df)

# Add the 'pd' records to 'gdf'

# Assuming you have already created the 'gdf' DataFrame with a 'Location_ID' column
# and the 'pd_df' DataFrame with a 'Video' column containing base64 encoded videos

# Loop through 'pd_df' DataFrame and populate 'videos' column in 'gdf'
for index, row in pd_df.iterrows():
    video_base64 = row['Video']
    gdf.at[index, 'videos'] = video_base64

# Now the 'videos' column in the 'gdf' DataFrame will be filled with base64 encoded videos from the 'pd_df' DataFrame

# Print the 'gdf' DataFrame
gdf.head()


#import dependencies
import folium
import pandas as pd

# Create a map object using the GeoDataFrame
m = folium.Map()

# Define a function to create the HTML content for the pop-up
def create_popup_html(row):
    unique_id = row['unique_id']
    coordinates = row['coordinates']
    GIS = row['gis']
    plus_code = row['plus_code']
    
    popup_html = f'''
        <b>Unique ID:</b> {unique_id}<br>
        <b>Coordinates:</b> {coordinates}<br>
        <b>GIS:</b> {GIS}<br>
        <b>PLUS CODE:</b> {plus_code}<br>
        <a href="#" onclick="toggleNavigationPane('{unique_id}')">View more details</a>
    '''

    return popup_html

# Define the CSS and JavaScript code for the navigation pane
css_navigation_pane = '''
    <style>
    /* Custom CSS for the navigation pane */
    .navigation-ui {
        display: none;
        position: absolute;
        top: 80px; /* Adjust the top position as needed */
        left: 10px;
        padding: 10px;
        background-color: white;
        z-index: 1000;
    }
    </style>
'''

js_navigation_pane = '''
    <script>
    // JavaScript code for the navigation pane
    function toggleNavigationPane(uniqueId) {
        var navigationPane = document.getElementById('navigation-ui-' + uniqueId);
        if (navigationPane.style.display === 'none') {
            navigationPane.style.display = 'block';
        } else {
            navigationPane.style.display = 'none';
        }
    }
    </script>
'''

# Function to generate custom HTML for the navigation pane
def generate_navigation_ui(row):
    sno = row['sno']
    unique_id = row['unique_id']
    coordinates = row['coordinates']
    GIS = row['gis']
    plus_code = row['plus_code']
    type_of_item = row['type_of_item']
    status = row['status']
    
    ui_html = f'''
        <div id="navigation-ui-{unique_id}" class="navigation-ui">
            <h2>Navigation UI</h2>
            <p><b>Sno:</b> {sno}</p>
            <p><b>Unique ID:</b> {unique_id}</p>
            <p><b>Coordinates:</b> {coordinates}</p>
            <p><b>GIS:</b> {GIS}</p>
            <p><b>PLUS CODE:</b> {plus_code}</p>
            <p><b>Type of item:</b> {type_of_item}</p>
            <p><b>Status:</b> {status}</p>
        </div>
    '''
    
    return ui_html

# Create the navigation pane HTML content
selected_columns = ['sno', 'unique_id', 'coordinates', 'gis', 'plus_code', 'type_of_item', 'status']
selected_gdf = gdf[selected_columns]

# Iterate over the rows of the GeoDataFrame
for index, row in gdf.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    popup_html = create_popup_html(row)
    popup = folium.Popup(html=popup_html, max_width=250)

    marker = folium.Marker(location=[latitude, longitude], popup=popup)
    marker.add_to(m)

    navigation_ui_html = generate_navigation_ui(row)
    navigation_ui_element = folium.Element(navigation_ui_html)
    m.get_root().html.add_child(navigation_ui_element)

    # Add a script tag to hide the navigation pane initially
    m.get_root().html.add_child(folium.Element(f"<script>document.getElementById('navigation-ui-{row['unique_id']}').style.display = 'none';</script>"))

# Add the CSS and JavaScript code to the map
m.get_root().html.add_child(folium.Element(css_navigation_pane))
m.get_root().html.add_child(folium.Element(js_navigation_pane))




# Save the map to an HTML file
m.save("templates/dummy_map.html")

m

import folium
import pandas as pd

# Sample GeoDataFrame with data
gdf = pd.DataFrame({
    'latitude': [51.5074, 40.7128, 34.0522],
    'longitude': [-0.1278, -74.0060, -118.2437],
    'sno': [1, 2, 3],
    'UniqueID': ['ID1', 'ID2', 'ID3'],
    'coordinates': ['51.5074, -0.1278', '40.7128, -74.0060', '34.0522, -118.2437'],
    'GIS': ['GIS1', 'GIS2', 'GIS3'],
    'PLUS CODE': ['CODE1', 'CODE2', 'CODE3'],
    'Nearby Location': ['Location1', 'Location2', 'Location3'],
    'Type of item': ['Type1', 'Type2', 'Type3'],
    'Status': ['Status1', 'Status2', 'Status3']
})

# Create a map object using the GeoDataFrame
m = folium.Map()

# Define the CSS and JavaScript code for the tooltip
css_tooltip = '''
/* Custom CSS for the tooltip */
.tooltip-col:hover .tooltiptext3,
.tooltip-col .tooltiptext3:hover {
    visibility: visible;
    opacity: 1;
}
.tooltip-col .tooltiptext3 {
    visibility: hidden;
    opacity: 0;
    transition: visibility 0s, opacity 0.3s linear;
    background-color: rgba(255, 255, 255, 0.7);
    border-radius: 6px;
    padding: 5px 10px;
    position: absolute;
    z-index: 1;
    top: -55px;
    left: -125px;
    width: 250px;
}
.tooltip-col .tooltiptext3::after {
    content: "";
    position: absolute;
    top: 100%;
    left: 50%;
    margin-left: -5px;
    border-width: 5px;
    border-style: solid;
    border-color: transparent transparent rgba(255, 255, 255, 0.7) transparent;
}

'''

# Define a function to create the HTML content for the tooltip
def create_tooltip_html(row):
    unique_id = row['UniqueID']
    coordinates = row['coordinates']
    GIS = row['GIS']
    plus_code = row['PLUS CODE']
    
    tooltip_html = f'''
        <b>Unique ID:</b> {unique_id}<br>
        <b>Coordinates:</b> {coordinates}<br>
        <b>GIS:</b> {GIS}<br>
        <b>PLUS CODE:</b> {plus_code}<br>
        <a href="#" onclick="toggleNavigationPane('{unique_id}', event)">View More Details</a>
    '''

    return tooltip_html


# Define the CSS and JavaScript code for the navigation pane
css_navigation_pane = '''
    <style>
    /* Custom CSS for the navigation pane */
    .navigation-ui {
        display: none;
        position: absolute;
        top: 80px; /* Adjust the top position as needed */
        left: 10px;
        padding: 10px;
        background-color: white;
        z-index: 1000;
    }
    </style>
'''

js_navigation_pane = '''
    <script>
    // JavaScript code for the navigation pane
    function toggleNavigationPane(uniqueId, event) {
        event.stopPropagation(); // Prevent the tooltip from closing

        var navigationPane = document.getElementById('navigation-ui-' + uniqueId);
        if (navigationPane.style.visibility === 'hidden' || navigationPane.style.opacity === '0') {
            navigationPane.style.visibility = 'visible';
            navigationPane.style.opacity = '1';
        } else {
            navigationPane.style.visibility = 'hidden';
            navigationPane.style.opacity = '0';
        }
    }
    </script>
'''

# Function to generate custom HTML for the navigation pane
def generate_navigation_ui(row):
    sno = row['sno']
    unique_id = row['UniqueID']
    coordinates = row['coordinates']
    GIS = row['GIS']
    plus_code = row['PLUS CODE']
    type_of_item = row['Type of item']
    status = row['Status']
    
    ui_html = f'''
        <div id="navigation-ui-{unique_id}" class="navigation-ui">
            <h2>Navigation UI</h2>
            <p><b>Sno:</b> {sno}</p>
            <p><b>Unique ID:</b> {unique_id}</p>
            <p><b>Coordinates:</b> {coordinates}</p>
            <p><b>GIS:</b> {GIS}</p>
            <p><b>PLUS CODE:</b> {plus_code}</p>
            <p><b>Type of item:</b> {type_of_item}</p>
            <p><b>Status:</b> {status}</p>
        </div>
    '''
    
    return ui_html

# Iterate over the rows of the GeoDataFrame
for index, row in gdf.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    tooltip_html = create_tooltip_html(row)

    marker = folium.Marker(location=[latitude, longitude], tooltip=tooltip_html, sticky=True)
    marker.add_to(m)

    navigation_ui_html = generate_navigation_ui(row)
    navigation_ui_element = folium.Element(navigation_ui_html)
    m.add_child(navigation_ui_element)

    # Add a script tag to hide the navigation pane initially
    m.get_root().html.add_child(folium.Element(f"<script>document.getElementById('navigation-ui-{row['UniqueID']}').style.visibility = 'hidden'; document.getElementById('navigation-ui-{row['UniqueID']}').style.opacity = '0';</script>"))

# Add the CSS and JavaScript code to the map
m.get_root().html.add_child(folium.Element(css_navigation_pane))
m.get_root().html.add_child

# Save the map to an HTML file
m.save('map_with_tooltip.html')

m


gdf.head()

import folium
import pandas as pd
import base64

# Sample GeoDataFrame with data
# Assuming you already have 'gdf' DataFrame as shown in the dataset

# Define a dictionary to map each type to a color
type_colors = {
    'A': 'blue',
    'B': 'green',
    'C': 'red'
}

# Create a map object using the GeoDataFrame
m = folium.Map()

# Define a function to create the HTML content for the pop-up pane
def create_popup_html(row):
    unique_id = row['unique_id']
    coordinates = row['coordinates']
    gis = row['gis']
    plus_code = row['plus_code']
    image_base64 = row['photos']
    
    # Specify the base64 encoded image source
    image_src = f'data:image/jpeg;base64,{image_base64}'
    
    popup_html = f'''
        <img src="{image_src}" alt="Image" width="200"><br>
        <b>Unique ID:</b> {unique_id}<br>
        <b>Coordinates:</b> {coordinates}<br>
        <b>GIS:</b> {gis}<br>
        <b>PLUS CODE:</b> {plus_code}<br>
        <a href="#" onclick="toggleNavigationPane('{unique_id}', event)">View More Details</a>
    '''

    return popup_html

# Define the CSS and JavaScript code for the navigation pane
css_navigation_pane = '''
    <style>
    /* Custom CSS for the navigation pane */
    .navigation-ui {
        display: none;
        position: absolute;
        top: 80px; /* Adjust the top position as needed */
        left: 10px;
        padding: 10px;
        background-color: white;
        z-index: 1000;
    }
    .video-container {
        margin-bottom: 10px;
    }
    </style>
'''

js_navigation_pane = '''
    <script>
    // JavaScript code for the navigation pane
    function toggleNavigationPane(uniqueId, event) {
        event.preventDefault();
        var navigationPane = document.getElementById('navigation-ui-' + uniqueId);
        if (navigationPane.style.display === 'none') {
            navigationPane.style.display = 'block';
        } else {
            navigationPane.style.display = 'none';
        }
    }
    </script>
'''

# Function to generate custom HTML for the navigation pane
def generate_navigation_ui(row):
    unique_id = row['unique_id']
    type_of_item = row['type_of_item']
    status = row['status']
    video_base64 = row['videos']
    
    # Specify the base64 encoded video source
    video_src = f'data:video/mp4;base64,{video_base64}'
    
    ui_html = f'''
        <div id="navigation-ui-{unique_id}" class="navigation-ui">
            <h2>Navigation UI</h2>
            <p><b>Unique ID:</b> {unique_id}</p>
            <p><b>Type of item:</b> {type_of_item}</p>
            <p><b>Status:</b> {status}</p>
            <div class="video-container">
                <video width="320" height="240" controls>
                    <source src="{video_src}" type="video/mp4">
                    Your browser does not support the video tag.
                </video>
            </div>
        </div>
    '''
    
    return ui_html

# Create the navigation pane HTML content
selected_columns = ['unique_id', 'type_of_item', 'status', 'videos']
selected_gdf = gdf[selected_columns]

# Iterate over the rows of the GeoDataFrame
for index, row in gdf.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    type_of_item = row['type_of_item']
    
    # Get the color for the marker based on the type
    marker_color = type_colors.get(type_of_item, 'gray')

    popup_html = create_popup_html(row)

    marker = folium.Marker(location=[latitude, longitude], popup=folium.Popup(popup_html),
                           icon=folium.Icon(color=marker_color), sticky=True)
    marker.add_to(m)

    navigation_ui_html = generate_navigation_ui(row)
    navigation_ui_element = folium.Element(navigation_ui_html)
    m.get_root().html.add_child(navigation_ui_element)

    # Add a script tag to hide the navigation pane initially
    m.get_root().html.add_child(folium.Element(f"<script>document.getElementById('navigation-ui-{row['unique_id']}').style.display = 'none';</script>"))

# Add the CSS and JavaScript code to the map
m.get_root().html.add_child(folium.Element(css_navigation_pane))
m.get_root().html.add_child(folium.Element(js_navigation_pane))

# Save the map to an HTML file
m.save('map.html')
